## Proyecto final BOOTCAMP IRONHACK
###### Estrategias de e-COMERCE

COMPANY = GYMSHARK

### PASO 3. ETL
- website : https://eu.gymshark.com/

In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

### DF Nº1: SALES 2023

In [107]:
# Importo el CSV de GYMSHARK SALES resultante del fake
sales_gymshark = pd.read_csv('sales_gymshark.concat.csv')

In [108]:
len(sales_gymshark)

410000

In [109]:
sales_gymshark.shape

(410000, 12)

In [110]:
sales_gymshark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410000 entries, 0 to 409999
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    410000 non-null  int64  
 1   StockCode    410000 non-null  object 
 2   Description  410000 non-null  object 
 3   Size         410000 non-null  object 
 4   Quantity     410000 non-null  int64  
 5   InvoiceDate  410000 non-null  object 
 6   CustomerID   410000 non-null  int64  
 7   Country      410000 non-null  object 
 8   Rating       379996 non-null  float64
 9   Colour       409078 non-null  object 
 10  Price        410000 non-null  float64
 11  Category     410000 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 37.5+ MB


In [111]:
sales_gymshark.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Size', 'Quantity',
       'InvoiceDate', 'CustomerID', 'Country', 'Rating', 'Colour', 'Price',
       'Category'],
      dtype='object')

In [112]:
sales_gymshark.isnull().sum()

InvoiceNo          0
StockCode          0
Description        0
Size               0
Quantity           0
InvoiceDate        0
CustomerID         0
Country            0
Rating         30004
Colour           922
Price              0
Category           0
dtype: int64

In [113]:
# RELLENAR NULOS DE LAS COLUMNAS "Rating" Y "Colour"

# Rellenar valores nulos en la columna "Rating" con 0, son productos que nunca ha sido rankeados por los clientes.
sales_gymshark['Rating'] = sales_gymshark['Rating'].fillna(0)

# Rellenar valores nulos en la columna "Colour" con "multicolored"
sales_gymshark['Colour'] = sales_gymshark['Colour'].fillna('multicolored')


In [114]:
sales_gymshark.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Size           0
Quantity       0
InvoiceDate    0
CustomerID     0
Country        0
Rating         0
Colour         0
Price          0
Category       0
dtype: int64

In [115]:
sales_gymshark.duplicated().sum()

0

In [116]:
# Convertir 'InvoiceDate' a formato de fecha y hora
sales_gymshark['InvoiceDate'] = pd.to_datetime(sales_gymshark['InvoiceDate'], format='%m-%d-%Y %A %H:%M')

# Separar la columna 'InvoiceDate' en fecha y hora
sales_gymshark['Day'] = sales_gymshark['InvoiceDate'].dt.day
sales_gymshark['Month'] = sales_gymshark['InvoiceDate'].dt.month
sales_gymshark['Year'] = sales_gymshark['InvoiceDate'].dt.year
sales_gymshark['Hour'] = sales_gymshark['InvoiceDate'].dt.strftime('%H:%M:%S')
sales_gymshark['DayOfWeek'] = sales_gymshark['InvoiceDate'].dt.dayofweek


# Mostrar las primeras filas
sales_gymshark.head()


,InvoiceNo,StockCode,Description,Size,Quantity,InvoiceDate,CustomerID,Country,Rating,Colour,Price,Category,Day,Month,Year,Hour,DayOfWeek
0,887,80691I,Heavyweight 1/4 Zip,XXS,1,2022-09-11 14:53:00,23521,Canada,0.0,Camo Brown,25.089517,Women,11,9,2022,14:53:00,6
1,416,13669R,"Sport 5"" Shorts",M,1,2022-02-20 23:54:00,56786,UK,0.0,Black Marl,10.404078,Women,20,2,2022,23:54:00,6
2,130,55386H,Arrival Woven Joggers,L,2,2022-04-26 02:17:00,59848,Germany,4.1,Stone Grey,14.879483,Men,26,4,2022,02:17:00,1
3,241,58064D,Built Graphic Joggers,L,3,2022-11-05 05:07:00,7797,UK,1.0,Force Green/Faded Green Marl,12.744996,Women,5,11,2022,05:07:00,5
4,405,12602O,Studio Tank,M,1,2022-12-17 16:00:00,38795,USA,4.2,Denim Teal,11.989461,Women,17,12,2022,16:00:00,5


### PROCEDO A CREAR COLUMNAS QUE NECESITO PARA ANALISIS:

In [117]:
# Eliminar comas como separadores de miles en las columnas 'Quantity' y 'Price'
sales_gymshark['Quantity'] = sales_gymshark['Quantity'].astype(str).str.replace(',', '')
sales_gymshark['Price'] = sales_gymshark['Price'].astype(str).str.replace(',', '')

# Convertir las columnas 'Price' a tipo de dato float
# Convertir la columna 'Quantity' a tipo de datos entero
sales_gymshark['Quantity'] = sales_gymshark['Quantity'].astype(int)

sales_gymshark['Price'] = sales_gymshark['Price'].astype(float)

# Realizar la multiplicación y redondear el resultado a dos decimales
sales_gymshark['Total'] = (sales_gymshark['Quantity'] * sales_gymshark['Price']).round(2)

# Convertir nuevamente a formato con coma como separador de miles y punto como separador decimal
sales_gymshark['Total'] = sales_gymshark['Total'].apply(lambda x: '{:,.2f}'.format(x))

# Reemplazar el punto por coma en la columna 'Rating'
sales_gymshark['Rating'] = sales_gymshark['Rating'].astype(str).str.replace('.', ',')

# Eliminar comas como separadores de miles en la columna 'Total'
sales_gymshark['Total'] = sales_gymshark['Total'].str.replace(',', '')

# Convertir la columna 'Total' a tipo de datos float
sales_gymshark['Total'] = sales_gymshark['Total'].astype(float)

# Reemplazar el punto por coma en la columna 'Rating'
sales_gymshark['Rating'] = sales_gymshark['Rating'].astype(str).str.replace('.', ',')


In [120]:
# Calcular el coste, la utilidad y los gastos operacionales
sales_gymshark['Total ingresos'] = sales_gymshark['Quantity'] * sales_gymshark['Price'].round(2)
sales_gymshark['Coste'] = (sales_gymshark['Total ingresos'] * 0.35).round(2)
sales_gymshark['Utilidad'] = (sales_gymshark['Total ingresos'] * 0.13).round(2)
sales_gymshark['Gastos Operacionales'] = (sales_gymshark['Total ingresos'] * 0.52).round(2)

# Reemplazar el punto por coma en las columnas 'Total', 'Coste', 'Utilidad' y 'Gastos Operacionales'
sales_gymshark['Total'] = sales_gymshark['Total'].astype(str).str.replace('.', ',')
sales_gymshark['Coste'] = sales_gymshark['Coste'].astype(str).str.replace('.', ',')
sales_gymshark['Utilidad'] = sales_gymshark['Utilidad'].astype(str).str.replace('.', ',')
sales_gymshark['Gastos Operacionales'] = sales_gymshark['Gastos Operacionales'].astype(str).str.replace('.', ',')


In [121]:
sales_gymshark

,InvoiceNo,StockCode,Description,Size,Quantity,InvoiceDate,CustomerID,Country,Rating,Colour,...,Day,Month,Year,Hour,DayOfWeek,Total,Total ingresos,Coste,Utilidad,Gastos Operacionales
0,887,80691I,Heavyweight 1/4 Zip,XXS,1,2022-09-11 14:53:00,23521,Canada,"0,0",Camo Brown,...,11,9,2022,14:53:00,6,"25,09",25.09,"8,78","3,26","13,05"
1,416,13669R,"Sport 5"" Shorts",M,1,2022-02-20 23:54:00,56786,UK,"0,0",Black Marl,...,20,2,2022,23:54:00,6,"10,4",10.40,"3,64","1,35","5,41"
2,130,55386H,Arrival Woven Joggers,L,2,2022-04-26 02:17:00,59848,Germany,"4,1",Stone Grey,...,26,4,2022,02:17:00,1,"29,76",29.76,"10,42","3,87","15,48"
3,241,58064D,Built Graphic Joggers,L,3,2022-11-05 05:07:00,7797,UK,"1,0",Force Green/Faded Green Marl,...,5,11,2022,05:07:00,5,"38,23",38.22,"13,38","4,97","19,87"
4,405,12602O,Studio Tank,M,1,2022-12-17 16:00:00,38795,USA,"4,2",Denim Teal,...,17,12,2022,16:00:00,5,"11,99",11.99,"4,2","1,56","6,23"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,770548,18747U,Lifting Club Long Sleeve T-Shirt,M,2,2023-05-14 11:06:00,80334,UK,"5,0",Willow Green/Desert Sage Green,...,14,5,2023,11:06:00,6,"49,14",49.14,"17,2","6,39","25,55"
409996,978199,86851C,Arrival 1/4 Zip,S,2,2023-11-03 21:43:00,97407,USA,"4,5",Ozone Blue,...,3,11,2023,21:43:00,4,"41,18",41.18,"14,41","5,35","21,41"
409997,123262,13022Z,Training Zip Hoodie,S,1,2023-12-19 20:51:00,53154,Canada,"3,9",Darkest Teal,...,19,12,2023,20:51:00,1,"37,12",37.12,"12,99","4,83","19,3"
409998,500449,29148V,Running 1/4 Zip,M,2,2023-10-05 07:22:00,12931,UK,"4,3",White,...,5,10,2023,07:22:00,3,"90,09",90.08,"31,53","11,71","46,84"


In [122]:
sales_gymshark.to_csv('sales_gymshark_clean2.csv', index=False)

### DF Nº 2: CUSTOMERS 2023

In [15]:
# Importo el CSV de CUSTOMER resultante del fake
customer = pd.read_csv('customer.csv')

In [16]:
len(customer)

100000

In [17]:
customer.shape

(100000, 6)

In [18]:
customer.columns

Index(['CustomerID', 'Name', 'Lastname', 'Age', 'Gender', 'Country'], dtype='object')

In [19]:
customer.isnull().sum()

CustomerID    0
Name          0
Lastname      0
Age           0
Gender        0
Country       0
dtype: int64

In [20]:
customer.duplicated().sum()

0